### 🎧 Recommendation Models with User Feedback Value (User Factors)

In this activity, we'll explore how to build personalized recommendation models based on **explicit user feedback**, also known as **user factors**.

This approach is similar to using **linear regression** to fill in missing ratings. However, instead of matrix factorization, we use individual regression models per artist, treating each artist's ratings as the target variable.

---

#### 💡 Scenario:

When users sign up for your streaming service, they are asked to rate a few core attributes — such as:

- `slick` 🎶 (e.g., production quality or polish)
- `lofi` 🎧 (e.g., rawness or chill factor)

These scores reflect a user’s musical preferences and act as **input features** to our model.

---

#### 🎯 Objective:

Use the `slick` and `lofi` scores provided by users to build **artist-specific regression models** that can:

- Predict how a user might rate an artist they haven’t listened to yet
- Fill in missing values in the user-item matrix
- Power personalized music recommendations

---

#### 🛠️ Approach:

1. For each artist (target column):
   - Use only users who rated that artist
   - Train a **Linear Regression** model using `slick` and `lofi` as inputs
2. Use the model to **predict missing ratings** for users who haven't rated that artist
3. Populate a complete user-artist rating matrix for recommendation

This method leverages **user-side data** to generalize across different artists, even if some ratings are missing.


In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression


#### The Data

Below, the data is loaded and displayed.  The `slick` and `lofi` columns contain user input values for their preferences accordingly.  

In [22]:
reviews = pd.read_csv('data/user_rated.csv', index_col = 0)
reviews

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B,slick,lofi
Alfred,3.0,4.0,NaN,4.0,4.0,5,5
Mandy,NaN,9.0,NaN,3.0,8.0,7,4
Lenny,2.0,5.0,8.0,9.0,NaN,3,6
Joan,3.0,NaN,9.0,4.0,9.0,5,5
Tino,1.0,1.0,NaN,9.0,5.0,1,8


#### Build Model for One Artist

In [27]:
non_null_filter = reviews['Michael Jackson'].isna() == False
null_filter = reviews['Michael Jackson'].isna()

X = reviews[non_null_filter][['slick', 'lofi']]
y = reviews[non_null_filter]['Michael Jackson']
mj_lr = LinearRegression().fit(X, y)
X_new = reviews[null_filter][['slick', 'lofi']]
mandy_predict = mj_lr.predict(X_new)

df_mandy = reviews.copy()
df_mandy.loc[null_filter, 'Michael Jackson'] = mandy_predict

### ANSWER CHECK
pd.DataFrame(mandy_predict.reshape(1,1), columns=["Mandy's Predicted Rating"], index=["Michael Jackson"])

,Mandy's Predicted Rating
Michael Jackson,4.0


#### Compute missing recommendation for each user

In [28]:
reviews

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B,slick,lofi
Alfred,3.0,4.0,NaN,4.0,4.0,5,5
Mandy,NaN,9.0,NaN,3.0,8.0,7,4
Lenny,2.0,5.0,8.0,9.0,NaN,3,6
Joan,3.0,NaN,9.0,4.0,9.0,5,5
Tino,1.0,1.0,NaN,9.0,5.0,1,8


In [32]:
df_full = reviews.copy()

for col in df_full.columns:
    if col not in ['slick', 'lofi']:
        non_null_filter = df_full[col].isna() == False
        null_filter = df_full[col].isna()
        # X
        X = df_full[non_null_filter][['slick','lofi']]
        # y
        y = df_full[non_null_filter][col]
        # X_new
        X_new = df_full[null_filter][['slick','lofi']]
        if not X_new.empty:
            lr = LinearRegression().fit(X, y)
            prediction = lr.predict(X_new)
            df_full.loc[null_filter, col] = prediction

df_full

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B,slick,lofi
Alfred,3.0,4.0,9.0,4.0,4.0,5,5
Mandy,4.0,9.0,10.0,3.0,8.0,7,4
Lenny,2.0,5.0,8.0,9.0,5.0,3,6
Joan,3.0,6.0,9.0,4.0,9.0,5,5
Tino,1.0,1.0,6.8,9.0,5.0,1,8
